# Тестовое задание

## Задание №1.
В данном блоке заданий у Вас имеются 2 таблицы:

CLIENTS:
Client_id	Client_Type	Client Address	Client_FIO
1	Пенсионер	Г.Москва, ул. Строителей, 3-2-234	Иванов И.И
2	Учащийся	Г.Подольск, ул. Ленина, 13	Петров П.П.
3	Работающий	Г.Пенза, ул. Василевского, 15	Кирилов К.К.
4	Работ. Пенсионер	Р.п. Лунино, ул. Главная, 18	Сидоров С.С.

DOCUMENTS:
Client_id	Valid_from	Valid_to	Document_type	Document_no	Income
1	2010-01-01	2010-12-01	Паспорт	111111-11	10 000
1	2012-12-01	2012-06-01	Паспорт	111111-11	11 000
1	2012-06-01	2220-01-01	Паспорт	333333-33	11 000
2	2001-01-01	2220-01-01	Паспорт	123123-44	5 000


В таблице с документами есть версионность, т.е. мы храним всю историю документов, которая у нас есть. Например, у клиента с №1:
Client_id	Valid_from	Valid_to	Document_type	Document_no	Income
1	2010-01-01	2010-12-01	Паспорт	111111-11	10 000
1	2012-12-01	2012-06-01	Паспорт	111111-11	11 000
1	2012-06-01	2220-01-01	Паспорт	333333-33	11 000
с 01.янв.2010 по 01.дек.2010 года был документ 111111-11, доход 10 000
с 01.дек.2010 по 01.июн.2012 документ не меняется, доход меняется 11 000
с 01.июн.2012 по текущий момент, документ стал 333333-33, доход не изменился, 11 000

Обратите внимание, 2220-01-01 – фиктивная, «бесконечно большая дата». По ней мы идентифицируем, текущую/последнюю версию документа.

In [15]:
import pandas as pd

In [11]:
import sqlite3
from datetime import date

# Подключаемся к базе (создаст файл, если нет)
conn = sqlite3.connect("clients.db")
cursor = conn.cursor()

# Удалим таблицы, если уже есть (для чистоты)
cursor.execute("DROP TABLE IF EXISTS DOCUMENTS;")
cursor.execute("DROP TABLE IF EXISTS CLIENTS;")

# Создание таблицы клиентов
cursor.execute("""
CREATE TABLE CLIENTS (
    Client_id INTEGER PRIMARY KEY,
    Client_Type TEXT,
    Client_Address TEXT,
    Client_FIO TEXT
);
""")

# Создание таблицы документов
cursor.execute("""
CREATE TABLE DOCUMENTS (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    Client_id INTEGER,
    Valid_from DATE,
    Valid_to DATE,
    Document_type TEXT,
    Document_no TEXT,
    Income NUMERIC,
    FOREIGN KEY (Client_id) REFERENCES CLIENTS(Client_id)
);
""")

# Данные клиентов
clients_data = [
    (1, "Пенсионер", "Г.Москва, ул. Строителей, 3-2-234", "Иванов И.И."),
    (2, "Учащийся", "Г.Подольск, ул. Ленина, 13", "Петров П.П."),
    (3, "Работающий", "Г.Пенза, ул. Василевского, 15", "Кирилов К.К."),
    (4, "Работ. Пенсионер", "Р.п. Лунино, ул. Главная, 18", "Сидоров С.С.")
]

cursor.executemany("""
INSERT INTO CLIENTS (Client_id, Client_Type, Client_Address, Client_FIO)
VALUES (?, ?, ?, ?);
""", clients_data)

# Данные документов (версионность)
documents_data = [
    # Клиент 1
    (1, "2010-01-01", "2010-12-01", "Паспорт", "111111-11", 10000),
    (1, "2010-12-01", "2012-06-01", "Паспорт", "111111-11", 11000),
    (1, "2012-06-01", "2220-01-01", "Паспорт", "333333-33", 11000),

    # Клиент 2
    (2, "2001-01-01", "2220-01-01", "Паспорт", "123123-44", 5000),

    # Клиент 3
    (3, "2015-05-01", "2220-01-01", "Паспорт", "555555-55", 25000),

    # Клиент 4
    (4, "2000-03-15", "2220-01-01", "Паспорт", "999999-99", 15000)
]

cursor.executemany("""
INSERT INTO DOCUMENTS (Client_id, Valid_from, Valid_to, Document_type, Document_no, Income)
VALUES (?, ?, ?, ?, ?, ?);
""", documents_data)

# Сохраняем и закрываем
conn.commit()
conn.close()

print("База данных clients.db успешно создана и заполнена.")

База данных clients.db успешно создана и заполнена.


In [13]:
conn = sqlite3.connect("clients.db")
cursor = conn.cursor()

1)	Напишите запрос, который вернет ФИО Клиентов по всем пенсионерам не из Москвы.

In [64]:
text_1 = """SELECT Client_FIO
            FROM CLIENTS
            WHERE Client_Type LIKE '%Пенсионер%' AND Client_Address NOT LIKE '%Г.Москва%';
         """

In [65]:
cursor.execute(text_1)
res = cursor.fetchall()
res = pd.DataFrame(res)
res

,0
0,Сидоров С.С.


2. Напишите запрос, который вернет: ФИО Клиентов, № документа (текущую/последнюю версию документа)

In [41]:
text_2 = """SELECT c.Client_FIO AS 'ФИО Клиента', d.Document_no AS '№ документа'
            FROM CLIENTS c LEFT JOIN DOCUMENTS d ON c.Client_id = d.Client_id
            WHERE d.Valid_to = '2220-01-01';
         """

In [42]:
cursor.execute(text_2)
res = cursor.fetchall()
res = pd.DataFrame(res)
res

,0,1
0,Иванов И.И.,333333-33
1,Петров П.П.,123123-44
2,Кирилов К.К.,555555-55
3,Сидоров С.С.,999999-99


3. Напишите запрос, который вернет: client_id, у которых в таблице с документами нет действующей версии (записи с valid_to=’2220-01-01’)

In [51]:
text_3 = """SELECT Client_id
            FROM CLIENTS
            WHERE Client_id NOT IN (SELECT Client_id
                                      FROM DOCUMENTS
                                      WHERE Valid_to = '2220-01-01');
         """

In [52]:
cursor.execute(text_3)
res = cursor.fetchall()
res = pd.DataFrame(res)
res

""


4)	Напишите запрос, который вернет: client_id, у которых в таблице с документами более одной действующей версии (записи с valid_to=’2220-01-01’)

In [53]:
text_4 = """SELECT Client_id
            FROM DOCUMENTS
            WHERE Valid_to = '2220-01-01'
            GROUP BY Client_id
            HAVING COUNT(*) > 1;
         """

In [54]:
cursor.execute(text_4)
res = cursor.fetchall()
res = pd.DataFrame(res)
res

""


5)	Напишите запрос, который вернет: ФИО Клиентов, у которых нет ни одной записи о документах.

In [57]:
text_5 = """SELECT c.Client_FIO AS 'ФИО Клиента'
            FROM CLIENTS c LEFT JOIN DOCUMENTS d ON c.Client_id = d.Client_id
            WHERE d.Client_id IS NULL;
         """

In [58]:
cursor.execute(text_5)
res = cursor.fetchall()
res = pd.DataFrame(res)
res

""
